In [136]:
cd '/content/drive/MyDrive/2022_WeatherContest_Data/DataSet'

/content/drive/MyDrive/2022_WeatherContest_Data/DataSet


In [138]:

# 백병원 데이터
back = pd.read_csv('hospital_data/back_hospital_new.csv',index_col = 0 )

# 인구 원본데이터
pap = [0] * 5
for i in range(len(pap)):
  pap[i] = pd.read_csv('papulation_data/'+str(i+2012)+'.csv', encoding ='cp949')


new_columns = ['area',"Jan","Jan_남","Jan_여",
               "Feb","Feb_남","Feb_여","Mar","Mar_남","Mar_여",
               "Apr","Apr_남","Apr_여","May","May_남","May_여",
               "Jun","Jun_남","Jun_여","Jul","Jul_남","Jul_여",
               "Aug","Aug_남","Aug_여","Sep","Sep_남","Sep_여",
               "Oct","Oct_남","Oct_여","Nov","Nov_남","Nov_여",
               "Dec","Dec_남","Dec_여"]

for i in range(len(pap)):
  pap[i].columns = new_columns
  pap[i].drop(0,axis =0,inplace=True)
  pap[i].drop(["Jan",
               "Feb","Mar",
               "Apr","May",
               "Jun","Jul",
               "Aug","Sep",
               "Oct","Nov",
               "Dec"],axis =1,inplace=True)
  pap[i].reset_index(drop=True,inplace=True)

pretreatment = pd.DataFrame()

for i in range(5):
  year = 2012+i
  for row in range(18):
     for j in range(1,25, 2):
      month = int((j+1)/2)
      area = pap[i].iloc[row,0]
      yymm = ''
      if month < 10:
        yymm = str(year) + '0' + str(month)
      else:
        yymm = str(year) + str(month)
      new_data = pd.DataFrame({
                'area':[area,area],
                'sex':[1,2],
                'yymmdd':[yymm,yymm],
                'tot_person':list(pap[i].iloc[row,j:j+2])})
      pretreatment = pd.concat([pretreatment,new_data])

pretreatment.reset_index(drop=True,inplace=True)
drop_row = pretreatment[pretreatment['area'] == '전국'].index
pretreatment.drop(drop_row ,axis=0,inplace=True)
pretreatment.reset_index(drop=True,inplace=True)

back.columns = ['yyyymmdd','area','sex','frequency']
old_name = pretreatment.area.unique()
new_name = back['area'].unique()

for i in range(len(old_name)):
  pretreatment.loc[pretreatment['area'] == old_name[i], 'area'] = new_name[i]

pretreatment_2 = pd.DataFrame()

def dd_insert(tot_date , i):
  global pretreatment_2
  for j in range(1,tot_date+1):
      date = ''
      if j<10:
        date = '0'+str(j)
      else:
        date = str(j)
      new_data_frame = pd.DataFrame({
          "area":list(pretreatment.iloc[i:i+2,0]),
          "sex":list(pretreatment.iloc[i:i+2,1]),
          "yymmdd":[str(pretreatment.iloc[i,2])+date,str(pretreatment.iloc[i,2])+date],
          "tot_person":list(pretreatment.iloc[i:i+2,3]),
      })
      pretreatment_2 = pd.concat([pretreatment_2,new_data_frame])


for i in range(0,len(pretreatment),2):
  month = int(pretreatment.iloc[i,2]) % 100
  if(month == 1 or month == 3 or month == 5 or month == 7 or month == 8 or month == 10 or month == 12):
    dd_insert(31,i)
  elif (month == 4 or month == 6 or month == 9 or month == 11):
    dd_insert(30,i)
  else:
    year = int(int(pretreatment.iloc[i,2])/100)
    if(year == 2012 or year == 2016):
      dd_insert(29,i)
    else:
      dd_insert(28,i)

pretreatment_2.columns = ['area', 'sex', 'yyyymmdd', 'tot_person']
pretreatment_2= pretreatment_2.astype({'yyyymmdd':'int64'})

back_merge_papulation = pd.merge(back,pretreatment_2, on=['yyyymmdd','area','sex'],how='outer')


del pretreatment_2
del pretreatment

back_merge_papulation_cpy = back_merge_papulation[back_merge_papulation['area'] == '세종']
back_merge_papulation_cpy_m = back_merge_papulation_cpy[back_merge_papulation_cpy['sex'] == 1]
back_merge_papulation_cpy_w = back_merge_papulation_cpy[back_merge_papulation_cpy['sex'] == 2]
def fil_tot_per(df):
  df['tot_person'] = pd.to_numeric(df['tot_person'],errors='coerce')
  df['tot_person'] = df['tot_person'].interpolate(limit_direction='backward')
  return df
back_merge_papulation_cpy_m = fil_tot_per(back_merge_papulation_cpy_m)
back_merge_papulation_cpy_w = fil_tot_per(back_merge_papulation_cpy_w)
back_merge_papulation_cpy_2 = back_merge_papulation.copy()
back_merge_papulation_cpy = pd.concat([back_merge_papulation_cpy_m,back_merge_papulation_cpy_w])
back_merge_papulation_cpy_2 = pd.merge(back_merge_papulation_cpy_2,back_merge_papulation_cpy, on=['yyyymmdd','area','sex','frequency'],how='outer')
back_merge_papulation_cpy_2['tot_person_x'] = pd.to_numeric(back_merge_papulation_cpy_2['tot_person_x'],errors='coerce')
back_merge_papulation_cpy_2['tot_person_x'].fillna(0,inplace=True)
back_merge_papulation_cpy_2['tot_person_y'].fillna(0,inplace=True)
back_merge_papulation_cpy_2['tot_person'] = back_merge_papulation_cpy_2['tot_person_x'] + back_merge_papulation_cpy_2['tot_person_y'] 
back_merge_papulation_cpy_2.drop(['tot_person_x','tot_person_y'],axis=1,inplace=True)
back_merge_papulation = back_merge_papulation_cpy_2

# 위험연령구간 데이터
def risk_age_primary_processing_man(target):
  data_set = [0]*4

  data_set[0] = pd.read_csv('risk_age_range/'+str(target)+'01_'+str(target)+'03_연령별인구현황_월간.csv',encoding='cp949')
  data_set[1] = pd.read_csv('risk_age_range/'+str(target)+'04_'+str(target)+'06_연령별인구현황_월간.csv',encoding='cp949')
  data_set[2] = pd.read_csv('risk_age_range/'+str(target)+'07_'+str(target)+'09_연령별인구현황_월간.csv',encoding='cp949')
  data_set[3] = pd.read_csv('risk_age_range/'+str(target)+'10_'+str(target)+'12_연령별인구현황_월간.csv',encoding='cp949')

  data_set_cpy = [0]*4

  for i in range(4):
    data_set_cpy[i]=data_set[i].copy()

  important_col = []
  for i in range(113):
    if(data_set[0].columns[i] == '행정구역' or data_set[0].columns[i] ==str(target)+'년01월_남_연령구간인구수'or data_set[0].columns[i] ==str(target)+'년02월_남_연령구간인구수' or data_set[0].columns[i] ==str(target)+'년03월_남_연령구간인구수'):
      important_col.append(i)
  col_name = []

  for i in range(127):
    if(i in important_col):
      col_name.append('important_'+str(i))
    else:
      col_name.append(str(i))

  for i in range(len(data_set)):
    data_set[i].columns = col_name
  
  drop_col = data_set[0].columns

  for i in range(4):
    for j in range(127):
      if(str(j) in drop_col):
        data_set[i].drop(str(j),axis=1,inplace=True)

  data_set[0].columns = ['area','man_01','man_02','man_03']
  data_set[1].columns = ['area','man_04','man_05','man_06']
  data_set[2].columns = ['area','man_07','man_08','man_09']
  data_set[3].columns = ['area','man_10','man_11','man_12']

  target_df = pd.merge(data_set[0],data_set[1], on=['area'],how='outer')
  target_df = pd.merge(target_df,data_set[2], on=['area'],how='outer')
  target_df = pd.merge(target_df,data_set[3], on=['area'],how='outer')

  for j in range(len(target_df)):
    target_df.iloc[j,:] = target_df.iloc[j,:].str.replace('[,]','')
  return target_df

def risk_age_primary_processing_woman(target):
  data_set = [0]*4

  data_set[0] = pd.read_csv('risk_age_range/'+str(target)+'01_'+str(target)+'03_연령별인구현황_월간.csv',encoding='cp949')
  data_set[1] = pd.read_csv('risk_age_range/'+str(target)+'04_'+str(target)+'06_연령별인구현황_월간.csv',encoding='cp949')
  data_set[2] = pd.read_csv('risk_age_range/'+str(target)+'07_'+str(target)+'09_연령별인구현황_월간.csv',encoding='cp949')
  data_set[3] = pd.read_csv('risk_age_range/'+str(target)+'10_'+str(target)+'12_연령별인구현황_월간.csv',encoding='cp949')

  important_col = []
  for i in range(127):
    if(data_set[0].columns[i] == '행정구역' 
      or data_set[0].columns[i] ==str(target)+'년01월_여_연령구간인구수'
      or data_set[0].columns[i] ==str(target)+'년01월_여_45~49세'
      or data_set[0].columns[i] ==str(target)+'년01월_여_50~54세'
      or data_set[0].columns[i] ==str(target)+'년02월_여_연령구간인구수'
      or data_set[0].columns[i] ==str(target)+'년02월_여_45~49세'
      or data_set[0].columns[i] ==str(target)+'년02월_여_50~54세'
      or data_set[0].columns[i] ==str(target)+'년03월_여_연령구간인구수'
      or data_set[0].columns[i] ==str(target)+'년03월_여_45~49세'
      or data_set[0].columns[i] ==str(target)+'년03월_여_50~54세'):
      important_col.append(i)

  col_name = []

  for i in range(127):
    if(i in important_col):
      col_name.append('important_'+str(i))
    else:
      col_name.append(str(i))
  for i in range(len(data_set)):
    data_set[i].columns = col_name
  
  drop_col = data_set[0].columns


  for i in range(4):
    for j in range(127):
      if(str(j) in drop_col):
        data_set[i].drop(str(j),axis=1,inplace=True)
  for i in range(4):
    for j in range(len(data_set[i])):
      data_set[i].iloc[j,1:4] = data_set[i].iloc[j,1:4].str.replace('[,]','')
      data_set[i].iloc[j,4:7] = data_set[i].iloc[j,4:7].str.replace('[,]','')
      data_set[i].iloc[j,7:10] = data_set[i].iloc[j,7:10].str.replace('[,]','')
      data_set[i].iloc[j,1:4] = pd.to_numeric(data_set[i].iloc[j,1:4],errors='coerce')
      data_set[i].iloc[j,4:7] = pd.to_numeric(data_set[i].iloc[j,4:7],errors='coerce')
      data_set[i].iloc[j,7:10] = pd.to_numeric(data_set[i].iloc[j,7:10],errors='coerce')
  
  data_set[0]['area'] = data_set[0]['important_0']
  data_set[1]['area'] = data_set[1]['important_0']
  data_set[2]['area'] = data_set[2]['important_0']
  data_set[3]['area'] = data_set[3]['important_0']

  data_set[0]['woman_01'] = data_set[0]['important_30'] -data_set[0]['important_31'] - data_set[0]['important_32']
  data_set[0]['woman_02'] = data_set[0]['important_72'] - data_set[0]['important_73'] - data_set[0]['important_74']
  data_set[0]['woman_03'] = data_set[0]['important_114'] - data_set[0]['important_115'] - data_set[0]['important_116']
  data_set[1]['woman_04'] = data_set[1]['important_30'] -data_set[1]['important_31'] - data_set[1]['important_32']
  data_set[1]['woman_05'] = data_set[1]['important_72'] - data_set[1]['important_73'] - data_set[1]['important_74']
  data_set[1]['woman_06'] = data_set[1]['important_114'] - data_set[1]['important_115'] - data_set[1]['important_116']
  data_set[2]['woman_07'] = data_set[2]['important_30'] -data_set[2]['important_31'] - data_set[2]['important_32']
  data_set[2]['woman_08'] = data_set[2]['important_72'] - data_set[2]['important_73'] - data_set[2]['important_74']
  data_set[2]['woman_09'] = data_set[2]['important_114'] - data_set[2]['important_115'] - data_set[2]['important_116']
  data_set[3]['woman_10'] = data_set[3]['important_30'] -data_set[3]['important_31'] - data_set[3]['important_32']
  data_set[3]['woman_11'] = data_set[3]['important_72'] - data_set[3]['important_73'] - data_set[3]['important_74']
  data_set[3]['woman_12'] = data_set[3]['important_114'] - data_set[3]['important_115'] - data_set[3]['important_116']

  for i in range(4):
    data_set[i].drop(['important_0','important_30','important_31','important_32','important_72','important_73','important_74','important_114','important_115','important_116'],axis=1,inplace=True)
  
  target_df = pd.merge(data_set[0],data_set[1], on=['area'],how='outer')
  target_df = pd.merge(target_df,data_set[2], on=['area'],how='outer')
  target_df = pd.merge(target_df,data_set[3], on=['area'],how='outer')

  return target_df

risk_m = [0] * 5
risk_w = [0] * 5

for i in range(5):
  year = 2012+i
  risk_m[i] = risk_age_primary_processing_man(year)
  risk_w[i] = risk_age_primary_processing_woman(year)

def drop_row(df, idx):
  return df.drop(df.index[idx])

for i in range(len(risk_m)):
  risk_m[i] = drop_row(risk_m[i],0)
  risk_w[i] = drop_row(risk_w[i],0)

new_names = back['area'].unique()

for i in range(len(risk_m)):
  
  risk_m[i].sort_values(by='area',axis=0,inplace=True)
  old_names = risk_m[i]['area'].unique()
  for j in range(len(old_names)):
    risk_m[i].loc[risk_m[i]['area'] == old_names[j], 'area'] = new_names[j]
  del old_names
  risk_m[i].reset_index(drop=True,inplace=True)

  risk_w[i].sort_values(by='area',axis=0,inplace=True)
  old_names = risk_w[i]['area'].unique()
  for j in range(len(old_names)):
    risk_w[i].loc[risk_w[i]['area'] == old_names[j], 'area'] = new_names[j]
  del old_names
  risk_w[i].reset_index(drop=True,inplace=True)

pretreatment1_m = pd.DataFrame()

for i in range(len(risk_m)):
  year = str(i+2012)
  for row in range(len(new_names)):
    for month in range(1,12+1):
      yyyymm = year
      if(month < 10):
        yyyymm += '0' + str(month)
      else:
        yyyymm += str(month)
      new_data_frame = pd.DataFrame({
          "area": risk_m[i].iloc[row,0],
          "yyyymm": yyyymm ,
          "num_risk_age":risk_m[i].iloc[row,month:month+1]
     })
      pretreatment1_m = pd.concat([pretreatment1_m, new_data_frame])

pretreatment1_w = pd.DataFrame()

for i in range(len(risk_w)):
  year = str(i+2012)
  for row in range(len(new_names)):
    for month in range(1,12+1):
      yyyymm = year
      if(month < 10):
        yyyymm += '0' + str(month)
      else:
        yyyymm += str(month)
      new_data_frame = pd.DataFrame({
          "area": risk_w[i].iloc[row,0],
          "yyyymm": yyyymm ,
          "num_risk_age":risk_w[i].iloc[row,month:month+1]
     })
      pretreatment1_w = pd.concat([pretreatment1_w, new_data_frame])

pretreatment1_w.reset_index(drop=True,inplace=True)
pretreatment1_m.reset_index(drop=True,inplace=True)

pretreatment1_m['sex'] =1
pretreatment1_w['sex'] =2

pretreatment1 = pd.concat([pretreatment1_m,pretreatment1_w])
pretreatment1.sort_values(by=['area','yyyymm'],inplace=True)
pretreatment1['num_risk_age'] = pd.to_numeric(pretreatment1['num_risk_age'],errors='coerce')
pretreatment1['yyyymm'] = pd.to_numeric(pretreatment1['yyyymm'],errors='coerce')


pretreatment_2 = pd.DataFrame()

def dd_insert(tot_date , i):
  global pretreatment_2
  for j in range(1,tot_date+1):
      date = ''
      if j<10:
        date = '0'+str(j)
      else:
        date = str(j)
      new_data_frame = pd.DataFrame({
          "area":list(pretreatment1.iloc[i:i+2,0]),
          "yymmdd":[str(pretreatment1.iloc[i,1])+date,str(pretreatment1.iloc[i,1])+date],
          "sex":list(pretreatment1.iloc[i:i+2,3]),
          "num_risk_age":list(pretreatment1.iloc[i:i+2,2]),
      })
      pretreatment_2 = pd.concat([pretreatment_2,new_data_frame])
  

for i in range(0,len(pretreatment1),2):
  month = int(pretreatment1.iloc[i,1]) % 100
  if(month == 1 or month == 3 or month == 5 or month == 7 or month == 8 or month == 10 or month == 12):
    dd_insert(31,i)
  elif (month == 4 or month == 6 or month == 9 or month == 11):
    dd_insert(30,i)
  else:
    year = int(int(pretreatment1.iloc[i,1])/100)
    if(year == 2012 or year == 2016):
      dd_insert(29,i)
    else:
      dd_insert(28,i)
pretreatment_2.reset_index(drop=True,inplace=True)
pretreatment_2.columns =['area', 'yyyymmdd', 'sex', 'num_risk_age']

pretreatment_2= pretreatment_2.astype({'yyyymmdd':'int64'})
back_merge_papulation_riskAge = pd.merge(back_merge_papulation,pretreatment_2, on=['yyyymmdd','area','sex'],how='outer')

del pretreatment_2

back_merge_papulation_riskAge_cpy = back_merge_papulation_riskAge[back_merge_papulation_riskAge['area'] == '세종']
back_merge_papulation_riskAge_cpy_m = back_merge_papulation_riskAge_cpy[back_merge_papulation_riskAge_cpy['sex'] == 1]
back_merge_papulation_riskAge_cpy_w = back_merge_papulation_riskAge_cpy[back_merge_papulation_riskAge_cpy['sex'] == 2]
def fil_tot_per(df):
  df['num_risk_age'] = pd.to_numeric(df['num_risk_age'],errors='coerce')
  df['num_risk_age'] = df['num_risk_age'].interpolate(limit_direction='backward')
  return df
back_merge_papulation_riskAge_cpy_m = fil_tot_per(back_merge_papulation_riskAge_cpy_m)
back_merge_papulation_riskAge_cpy_w = fil_tot_per(back_merge_papulation_riskAge_cpy_w)
back_merge_papulation_riskAge_cpy_2 = back_merge_papulation_riskAge.copy()
back_merge_papulation_riskAge_cpy = pd.concat([back_merge_papulation_riskAge_cpy_m,back_merge_papulation_riskAge_cpy_w])
back_merge_papulation_riskAge_cpy_2 = pd.merge(back_merge_papulation_riskAge_cpy_2,back_merge_papulation_riskAge_cpy, on=['yyyymmdd','area','sex','frequency','tot_person'],how='outer')
back_merge_papulation_riskAge_cpy_2['num_risk_age_x'] = pd.to_numeric(back_merge_papulation_riskAge_cpy_2['num_risk_age_x'],errors='coerce')
back_merge_papulation_riskAge_cpy_2['num_risk_age_x'].fillna(0,inplace=True)
back_merge_papulation_riskAge_cpy_2['num_risk_age_y'].fillna(0,inplace=True)
back_merge_papulation_riskAge_cpy_2['num_risk_age'] = back_merge_papulation_riskAge_cpy_2['num_risk_age_x'] + back_merge_papulation_riskAge_cpy_2['num_risk_age_y'] 
back_merge_papulation_riskAge_cpy_2.drop(['num_risk_age_x','num_risk_age_y'],axis=1,inplace=True)
back_merge_papulation_riskAge = back_merge_papulation_riskAge_cpy_2

## 최종 데이터 : back_merge_papulation_riskAge

,yyyymmdd,area,sex,frequency,tot_person,num_risk_age
0,20120101,강원,1,3.0,772718.0,322636.0
1,20120101,강원,2,3.0,762617.0,229447.0
2,20120101,경기,1,4.0,6020446.0,2054584.0
3,20120101,경기,2,5.0,5928150.0,1182747.0
4,20120101,경남,1,2.0,1665308.0,622268.0
...,...,...,...,...,...,...
62113,20161229,충북,2,NaN,788385.0,258676.0
62114,20161230,충북,1,NaN,803240.0,358361.0
62115,20161230,충북,2,NaN,788385.0,258676.0
62116,20161231,충북,1,NaN,803240.0,358361.0
